In [1]:
# def get_difference(list_a, list_b):
#     return set(list_a)-set(list_b)
 
# list_a = df1['title']
# list_b = df2['title']
 
# non_match = list(get_difference(list_a, list_b))
# print("No match elements: ", non_match)

In [1]:
import pandas as pd
from urllib.parse import urlparse

In [2]:
# Load and get shape to same format as export in deduplication notebook
df = pd.read_csv('/content/Labs28_AllSources_wo_DuplicateLinks2020-11-09 03_02_06.620162+00_00 (3).csv', sep="|")
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df['lat'] = df['lat'].index.astype(float)
df['long'] = df['long'].index.astype(float)
# Drop all unneccessary columns
df.drop(labels=['date', 'id', 'city', 'state', 'lat', 'long', 'title',
       'description', 'tags', 'verbalization', 'empty_hand_soft',
       'empty_hand_hard', 'less_lethal_methods', 'lethal_force',
       'uncategorized'], axis=1, inplace=True)

In [3]:
col = df.links

def stringtolinks(col):
    """
    Takes a column containing multiple urls as one string and parses urls
    """ 
    l, words, chars = [], [], []

    for row in range(len(col)):
        # l.append(words)
        # words =[]

        for char in col[row]:

            if (ord(char) != ord("'")) and (ord(char) != ord("[")) and (ord(char) != ord("]")) and (char != " "):

                    if char != ",":
                        chars.append(char)
                        
                    # elif char == ",":
                    else:
                        words.append("".join(chars))
                        chars = []

        words.append("".join(chars))  
        l.append(words)
        chars = []
        words =[]
    return l

df['testlinks'] = stringtolinks(col)


source, path = [], []

for i, row  in enumerate(df.testlinks):
    source_p, path_p = [], []

    for item in row:

        url = item
        o = urlparse(url)
        source_p.append(o.netloc)
        
        # source_p
        path_p.append(o.path)

    source.append(set(source_p))
    path.append(path_p)

df['source'], df['paths'] = source, path


col = df.paths
def splitusers(col):
    users = []
    user = set()
    chars = ""
    counter = 0

    for row in col:
        for item in row:
            for char in item:
                
                if char == "/" and counter != 2:
                    counter += 1 
                else:
                    chars = chars + char

                if counter == 2:
                    break
                    continue

            user.add(chars)
            # user.append(chars)
            chars = ""
            counter = 0

        users.append(user)
        user = set()
        chars = ""
        counter = 0

    return users
df['users'] = splitusers(col)
df.drop(labels=['links', 'paths'], axis=1, inplace=True)
df

,testlinks,source,users
0,"[https://youtu.be/XAa5xb6JitI?t=5982, https://...","{youtu.be, gfycat.com}",{distinctsecretgrasshopper-minneapolis-protest...
1,[https://www.facebook.com/damicedsota.thespiri...,{www.facebook.com},{damicedsota.thespiritflow}
2,[https://www.facebook.com/1462345700/posts/102...,{www.facebook.com},{1462345700}
3,[https://www.denverpost.com/2020/05/29/denver-...,"{www.denverpost.com, www.nytimes.com}",{2020}
4,[https://twitter.com/ShimonPro/status/12661365...,{twitter.com},{ShimonPro}
...,...,...,...
1243,[https://twitter.com/Cascadianphotog/status/13...,{twitter.com},"{hungrybowtie, ByMikeBaker, Cascadianphotog}"
1244,[https://twitter.com/onelung_/status/132420651...,"{twitter.com, www.youtube.com}","{watch, onelung_}"
1245,[https://twitter.com/protest_nyc/status/132415...,"{twitter.com, go.citizen.com}","{Kyle_McMorrow, protest_nyc, greg_doucette, U4..."
1246,[https://twitter.com/ByMikeBaker/status/132420...,"{twitter.com, mobile.twitter.com}","{hungrybowtie, ByMikeBaker, MrOlmos}"


In [4]:
url = df.testlinks[0][0]  #parse a single url
o = urlparse(url)
o.netloc, o.path

('youtu.be', '/XAa5xb6JitI')

In [15]:
df.iloc[0]

testlinks    [https://youtu.be/XAa5xb6JitI?t=5982, https://...
source                                  {youtu.be, gfycat.com}
users        {distinctsecretgrasshopper-minneapolis-protest...
Name: 0, dtype: object